## Model description

This model corresponds to the "classical" bouncing ball. It is perhaps the simplest example of a hybrid ODE, since it presents only one discrete transition. The model consists of a ball dropped from a predefined height. It hits the ground after a certain time, loses energy and then bounces back into the air and starts to fall again.

This model has 2 continuous variables, 1 mode and 1 discrete jump.

The physical phenomena can be represented as the following hybrid automaton:

<img src="BouncingBall.png" alt="Drawing" style="width: 300px;"/>

In the discrete state, the motion of the ball, assumed to have a mass of $m = 1 kg$, is governed by the following differential equation:

$$
\begin{array} \\x'(t)= & v(t)\\ v'(t) = & -9.81 \end{array}
$$
where $x$ is the ball’s height from the ground, $v$ is the ball’s vertical velocity, and $9.81ms^{-2}$ is the earth’s gravitational force.

The invariant $x \geq 0$ enforces that the ball always bounces when it reaches the ground. The guard $x=0 \wedge v\leq0$ of the single discrete transition, which models the bouncing, ensures that bouncing happens after falling when reaching the ground. The corresponding reset condition $v:=-c \cdot v$ accounts for the loss of energy due to the ball’s deformation, where $c\in\left[0,1\right]$ is a constant known as [coefficient of restitution](https://en.wikipedia.org/wiki/Coefficient_of_restitution).

The model parameters and description are taken from the [HyPro collection of continuous and hybrid system benchmarks](https://ths.rwth-aachen.de/research/projects/hypro/).

In [ ]:
using Revise # to debug
using Reachability, HybridSystems, MathematicalSystems, LazySets, LazySets.Approximations, LinearAlgebra 

In [ ]:
function bouncing_ball(;c = 0.75,
                       X0 = Interval(10.0, 10.2) × Interval(0, 0),
                       T = 40.0)

    # automaton structure
    automaton = LightAutomaton(1)

    # falling : x' = Ax + b, x ∈ X
    A = [0.0 1.0;
         0.0 0.0]
    b = [0.0,
        -9.81]
    X = HalfSpace([-1.0, 0.0], 0.0) # x >= 0
    falling = ConstrainedAffineContinuousSystem(A, b, X)
        
    # modes
    modes = [falling]

    # transition from mode falling to mode falling (self-loop)
    add_transition!(automaton, 1, 1, 1)
    
    # reset map (x, v) -> (x, -cv) with guard x == 0 and v <= 0
    A = [1.0 0.0;
         0.0 -c]
    
    guard = HPolyhedron([HalfSpace([0.0, 1.0], 0.0),   # v ≤ 0
                         HalfSpace([-1.0, 0.0], 0.0),  # x ≥ 0
                         HalfSpace([1.0, 0.0], 0.0)])  # x ≤ 0
    
    resetmaps = [ConstrainedLinearMap(A, guard)]

    # switching
    switchings = [AutonomousSwitching()]

    ℋ = HybridSystem(automaton, modes, resetmaps, switchings)

    # initial condition in "falling" mode
    initial_condition = [(1, X0)]

    problem = InitialValueProblem(ℋ, initial_condition)

    options = Options(:mode=>"reach", :T=>T, :plot_vars=>[1, 2])

    return (problem, options)
end

## Reachability settings

We consider the inital set given by an uncertain initial position and certain momentum (at rest):

$$
 x \in [10, 10.2], v = 0.
$$

We use a time horizon of $T=40s$ and set the constant $c = 0.75$.
The set of *bad states* is the set of all states where $v \geq 10.7$.

In [ ]:
# settings
c = 0.75
X0 = Interval(10.0, 10.2) × Interval(0, 0)
T = 20.0

BouncingBall, options = bouncing_ball(c=c, X0=X0, T=T);
BadStates = HalfSpace([0.0, -1.0], -10.7); # v >= 10.7

## Results

In [ ]:
using Plots, LaTeXStrings

In [ ]:
#=
doesn't work -- why?
@time begin
    sol = solve(BouncingBall, merge!(Options(:max_jumps=>0), options),
                BFFPSV18(:δ=>0.1, :block_options=>OctDirections));
end;
=#


#=
@time begin
    opts = merge!(Options(:max_jumps=>1, :project_reachset=>false, :verbosity=>1), options)
    opC = GLGM06(:δ=>0.1, :max_order=>2)
    opD = LazyDiscretePost(:check_invariant_intersection=>true,
                           :lazy_R⋂I=>false, :overapproximation=>OctDirections)
    
    sol = solve(BouncingBall, opts, opC, opD)
end;
MethodError: Cannot `convert` an object of type HPolytope{Float64} to an object of type Zonotope
=#

@time begin
    opts = merge!(Options(:max_jumps=>1, :project_reachset=>true, :set_type_proj=>HPolygon,
                          :verbosity=>0), options)
    opC = BFFPSV18(:δ=>0.1, :partition=>[1:2], :block_options=>OctDirections)
    opD = LazyDiscretePost(:check_invariant_intersection=>false,
                           :lazy_R⋂I=>true, :overapproximation=>OctDirections)
    sol = solve(BouncingBall, opts, opC, opD);
end;

In [ ]:
sol.options.dict

In [ ]:
sol.Xk[1].X, sol.Xk[2].X 

In [ ]:
plot(sol)

In [ ]:
plot(sol, xlab=L"x", ylab=L"v", alpha=.5)

In [ ]:
plot(sol, xlab=L"x", ylab=L"v", alpha=.5)

In [ ]:
# just checking the flowpipe for the "falling" mode:

In [ ]:
X0

In [ ]:
# falling : x' = Ax + b, x ∈ X
A = [0.0 1.0;
     0.0 0.0]

b = [0.0,
    -9.81]

problem = IVP(AffineContinuousSystem(A, b), X0)
options = Options(:mode=>"reach", :T=>10., :plot_vars=>[1, 2])

# produces box flowipe
sol = solve(problem, options, op=BFFPSV18(:δ=>1.0));

using LazySets.Approximations

# produces box flowipe -- not too accurate, compare w/zonotopes 
#sol = solve(problem, options, op=BFFPSV18(:δ=>0.1));

# should produce octagons -- doesn't work !
#sol = solve(problem, options, op=BFFPSV18(:δ=>0.01, :partition=>[1:2],
#            :block_options=>OctDirections));

# use zonotope-based algorithm
@time sol = solve(problem, options, op=GLGM06(:δ=>0.1, :max_order=>2));

In [ ]:
plot([sol.Xk[i].X for i in eachindex(sol.Xk)], alpha=.5, color=:lightblue)